a)
Write a function get_comic_info that gets data on a given comic as a dictionary. The function must accept
comic_number as an argument and return data for that comic. If the comic does not exist, the function must
return an empty dictionary. (Hint: Remember to check the if the response is ok, i.e status code less than 400.)
Below are examples of how the function should behave (the ordering of keys in the dict does not matter):

In [1]:
import requests

## A
Write a function get_comic_info that gets data on a given comic as a dictionary. The function must accept
comic_number as an argument and return data for that comic. If the comic does not exist, the function must
return an empty dictionary. (Hint: Remember to check the if the response is ok, i.e status code less than 400.)
Below are examples of how the function should behave (the ordering of keys in the dict does not matter):


In [4]:
def get_comic_info(COMICNO):
    response = requests.get(f"https://xkcd.com/{COMICNO}/info.0.json")

    if response.ok:
        return response.json()
    else: 
        print("There was something wrong with your response")
        return {}
    
 



In [5]:
comic1 = get_comic_info(1)
print(comic1)

comic2 = get_comic_info(20000)
print(comic2)

{'month': '1', 'num': 1, 'link': '', 'year': '2006', 'news': '', 'safe_title': 'Barrel - Part 1', 'transcript': "[[A boy sits in a barrel which is floating in an ocean.]]\nBoy: I wonder where I'll float next?\n[[The barrel drifts into the distance. Nothing else can be seen.]]\n{{Alt: Don't we all.}}", 'alt': "Don't we all.", 'img': 'https://imgs.xkcd.com/comics/barrel_cropped_(1).jpg', 'title': 'Barrel - Part 1', 'day': '1'}
There was something wrong with your response
{}


# B
You might have noticed that the returned dict in task a) contains a lot of keys. We are only interested in the
fields day, month, year, num and title, and we want the first four of these as int. Write a function
transform_comic that accepts a comic as input and returns a stripped down version of that comic. The
comic should be the dict from get_comic_info.


In [6]:
def transform_comic(comic_dict):
    transformed_dict = {}
    labels = ["day", "month", "year", "title"]
    if len(comic_dict) == 0:
        print("Comic dictonary is empty")
    else:
        transformed_dict["day"] = comic_dict["day"]
        transformed_dict["month"] = comic_dict["month"]
        transformed_dict["year"] = comic_dict["year"]
        transformed_dict["title"] = comic_dict["title"]

    return transformed_dict

transform_dict = transform_comic(comic1)

transform_dict




{'day': '1', 'month': '1', 'year': '2006', 'title': 'Barrel - Part 1'}

## c
Write a function get_all_comics that returns a list of comic data. It should use both get_comic_info and
transform_comic. The function takes two input parameters, num1 and num2, and the function returns the
data for all comics between num1 and num2 (inclusive, meaning there is data for comics num1 and num2 in the
list).


In [7]:
def get_all_comics(min_val, max_val):
    list_of_comic_data = []
    # Need +1 on range or else we get -1 of the values we enter
    for i in range(min_val, max_val+1):
        #I like functions so I put a function in a function
        list_of_comic_data.append(transform_comic(get_comic_info(i)))
    return list_of_comic_data

In [8]:
get_all_comics(1,5)

[{'day': '1', 'month': '1', 'year': '2006', 'title': 'Barrel - Part 1'},
 {'day': '1', 'month': '1', 'year': '2006', 'title': 'Petit Trees (sketch)'},
 {'day': '1', 'month': '1', 'year': '2006', 'title': 'Island (sketch)'},
 {'day': '1', 'month': '1', 'year': '2006', 'title': 'Landscape (sketch)'},
 {'day': '1', 'month': '1', 'year': '2006', 'title': 'Blown apart'}]

## d)
Write a function get_time_delta takes input comic1 and comic2, and returns the number of days it has
been between the publish date of the two comics. If either of the inputs are None, it should return 0.

In [9]:
from datetime import date

In [10]:
def get_time_delta(comic_dict1, comic_dict2):
    if comic_dict1 == None or comic_dict2 == None:
        return 0
    # Convert the data type to int otherwise math won't work
    date_one = date(int(comic_dict1["year"]), int(comic_dict1["month"]), int(comic_dict1["day"]))
    date_two = date(int(comic_dict2["year"]), int(comic_dict2["month"]), int(comic_dict2["day"]))
    difference = date_two - date_one

    return difference.days

In [11]:
comic1, comic2 = get_all_comics(513, 514) 


In [12]:
print(get_time_delta(comic1, comic2))
print(get_time_delta(None, comic2))

3
0


## e)
Write a function get_all_comics_with_time_delta that returns a list of comic data (as above with
get_all_comics). The function takes two input parameters, num1 and num2.
In addition to the fields returned above, each dictionary in the list should have the field timedelta, which
indicates the number of days since the previous comic. For the first comic in the list, the timedelta should be 0
(since we have nothing to compare it against).


In [13]:
def get_all_comics_with_time_delta(min_val, max_val):
    
    #this will be a list since get_all_comics returns the comics as a list
    list_of_comics = get_all_comics(min_val, max_val)

    #The Return value
    comic_data = []

    #New Key to make in the dictionary, in case it needs to be altered later
    TimeDeltaLabel = "timedelta"

    # Use enumerate to easily use the index to find previous comic
    for index, comic in enumerate(list_of_comics):
        if(index == 0):
            #The one case where the first comic doesn't have a previous comic
            comic[TimeDeltaLabel] = 0
        else:
            previous_comic = index-1
            #Use the index of the previous comic first as that will always have the min value
            comic[TimeDeltaLabel] = get_time_delta(list_of_comics[previous_comic],comic)

        #Add the time delta modified dictionary to a new list   
        comic_data.append(comic)
           
    return comic_data

In [14]:
print(get_all_comics_with_time_delta(513, 515))

[{'day': '5', 'month': '12', 'year': '2008', 'title': 'Friends', 'timedelta': 0}, {'day': '8', 'month': '12', 'year': '2008', 'title': 'Simultaneous', 'timedelta': 3}, {'day': '10', 'month': '12', 'year': '2008', 'title': 'No One Must Know', 'timedelta': 2}]


## f)
Write a function write_comics_data_with_time_delta that takes two input parameters, num1 and num2.
The function should write all comic data for these comics (as retrieved in the previous problems) to a CSV file
comics.csv. Use csv.DictWriter to achieve this.


In [15]:
import csv

In [17]:
def write_comics_data_with_time_delta(num1, num2):
    all_comics = get_all_comics_with_time_delta(num1, num2)
    csv_out_file_path = 'time_delta.csv'
    with open(csv_out_file_path, 'w', newline='') as csv_file:

                
        header = ["day","month","year","title","timedelta"]
        csv_writer = csv.DictWriter(csv_file, fieldnames=header)
        csv_writer.writeheader()

        # Write the data to the CSV file
        csv_writer.writerows(all_comics)

In [18]:
write_comics_data_with_time_delta(513, 518)